<img style="float: left; margin:0px 15px 15px 0px; width:120px" src="https://www.orfeo-toolbox.org/wp-content/uploads/2016/03/logo-orfeo-toolbox.png">

# OTB Guided Tour - JURSE 2019 Vannes France - May 21th
## Yannick TANGUY and David YOUSSEFI (CNES, French Space Agency)

<br>

<b> Press <span style="color:black;background:yellow">SHIFT+ENTER</span> to execute the notebook interactively cell by cell </b></div>

## Step 3 : Segment watermask thresholding NDWI

### Functions of the full OTB Pipeline

In [ ]:
import otbApplication

def app_radiometricindices(images):
    index = 0
    for im in images:
        # Here we create an application
        app = otbApplication.Registry.CreateApplication("BandMath")
        # BandMath takes a list of images as input,
        # so we have to give a Python list [ image ], or [ image1, image2, .., imageN]
        app.SetParameterStringList("il",[im])
        app.SetParameterString("out",im.replace("RVBPIR.tif","_indices.tif"))
        # the main parameter is the mathematical expression
        # here, we compute NDVI : (nir - red) / (nir + red)
        # The corresponding bands for NIR and Red are respectively the 4th and the 3rd bands (b4, b3)
        # of the first image (im1)
        app.SetParameterString("exp","(im1b2+im1b4) > 0 ? (im1b2-im1b4)/(im1b2+im1b4) : -1 ")
        app.Execute()
        if (index == 0):
            tab_App = [app]
        else:
            tab_App.append(app)
        index += 1
    return tab_App

def compute_watermask(list_app,outputfilename):
    app = otbApplication.Registry.CreateApplication("BandMath")

    index = 1
    input_list = []
    exp = "max("
    for appli in list_app:
        app.AddImageToParameterInputImageList("il",appli.GetParameterOutputImage("out"))
        if index==1:
            exp = exp+"im"+str(index)+"b1"
        else:
            exp = exp+", "+"im"+str(index)+"b1"
        index += 1

    exp = exp +")"
    print(exp)
    print(input_list)

    app.SetParameterString("out",outputfilename)
    app.SetParameterString("exp",exp)
    app.Execute()
    return app

def thresholdNDWI(appWatermask,outputfilename,threshold):
    app = otbApplication.Registry.CreateApplication("BandMath")
    app.AddImageToParameterInputImageList("il", appWatermask.GetParameterOutputImage("out"))
    app.SetParameterString("exp", "im1b1 > "+str(threshold)+" ? 1 : 0")
    app.SetParameterString("out", outputfilename)
    app.ExecuteAndWriteOutput()
    return app

def smoothResult(thresholdNDWI, radius, outputfilename):
    app = otbApplication.Registry.CreateApplication("BinaryMorphologicalOperation")
    app.SetParameterInputImage("in",thresholdNDWI.GetParameterOutputImage("out"))
    app.SetParameterString("out",outputfilename)
    app.SetParameterString("structype","ball")
    app.SetParameterFloat("structype.ball.xradius", radius)
    app.SetParameterFloat("structype.ball.yradius", radius)
    app.SetParameterString("filter","closing")
    app.ExecuteAndWriteOutput()


### Full OTB Pipeline

In [ ]:
import os

import utils

# Threshold
threshold = 0.3

# Data directory
DATA_DIR = "data"

# Output directory
OUTPUT_DIR = "output"

# Input / Output filenames
images_list = utils.list_images(DATA_DIR,"_RVBPIR.tif")
ndwi_max_fname = os.path.join(OUTPUT_DIR, "max_NDWI2.tif")
ndwi_thres_fname = os.path.join(OUTPUT_DIR, "ndwi_threshold%d.tif" % (threshold*100) )

# Compute radiometric indices on a stack of images
list_app = app_radiometricindices(images_list)

# Compute a water mask from NDWI2
wmask = compute_watermask(list_app, ndwi_max_fname)

# Use a threshold to extract the mask
ndwi = thresholdNDWI(wmask, ndwi_thres_fname,threshold)

# Smooth the result by applying a morphological operation
# smoothResult(ndwi,2.0,wdir+"/result_smoothed.tif")

print ("Full OTB Pipeline completed")

### Display OTB result

In [ ]:
import rasterio
import display_api

raster = rasterio.open(ndwi_thres_fname)
m, dc = display_api.rasters_on_map([raster], OUTPUT_DIR, ["OTB result"])
m

### Go to the [Step 4](step4_filter_shapefile.ipynb)